In [1]:
import torch
assert torch.__version__>='1.2.0', 'Expect PyTorch>=1.2.0 but get {}'.format(torch.__version__)
from torch import nn
import torch.nn.functional as F

import numpy as np

import time
import os
import sys
import pickle

imp_dir = '../Implementations'
sys.path.insert(1, imp_dir)
data_dir = '../Data/criteo'
sys.path.insert(1, data_dir)

from FM_BinClf_Torch import FM_2D_Layer

import logging
import importlib
importlib.reload(logging)

logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(levelname)-6s %(message)s', level=logging.INFO, datefmt='%H:%M:%S')
logger = logging.getLogger(__name__)

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info('Device in Use: {}'.format(DEVICE))

16:18:49 INFO   Device in Use: cuda


## Prepare Data

### List all available files

In [3]:
np_artifact_dir = os.path.join(data_dir, 'criteo_train_numpy_artifact')
index_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='index', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
value_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='value', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
label_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='label', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))

In [4]:
start = time.time()

train_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[:10]]),
)

logger.info('Training data loaded after {:.2f}s'.format(time.time()-start))

start = time.time()

test_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[10:]]),
)

logger.info('Test data loaded after {:.2f}s'.format(time.time()-start))

17:26:58 INFO   Training data loaded after 4088.81s
17:52:11 INFO   Test data loaded after 1512.96s


## Create Model

In [5]:
embedding_map_dict_pkl_path = os.path.join(data_dir, 'criteo_feature_dict_artifact/categorical_feature_map_dict.pkl')
with open(embedding_map_dict_pkl_path, 'rb') as f:
    embedding_map_dict = pickle.load(f)

In [6]:
FM = FM_2D_Layer(len(embedding_map_dict)+60,39,5).to(DEVICE)

## Sample Run

In [7]:
import execution
importlib.reload(execution)

<module 'execution' from '../Data/criteo/execution.py'>

In [8]:
cwd = os.getcwd()
checkpoint_dir = os.path.join(cwd, 'FM_artifact')
checkpoint_prefix = 'FM'

In [9]:
execution.train_model(FM, train_data, test_data, F.binary_cross_entropy_with_logits, torch.optim.Adam(FM.parameters()), DEVICE, checkpoint_dir, checkpoint_prefix)

17:52:54 INFO   Epoch 1/5 - Batch 1000/16000 Done - Train Loss: 0.466783, Val Loss: 0.458799
17:53:03 INFO   Epoch 1/5 - Batch 2000/16000 Done - Train Loss: 0.461234, Val Loss: 0.464833
17:53:13 INFO   Epoch 1/5 - Batch 3000/16000 Done - Train Loss: 0.458245, Val Loss: 0.454681
17:53:23 INFO   Epoch 1/5 - Batch 4000/16000 Done - Train Loss: 0.458022, Val Loss: 0.458335
17:53:32 INFO   Epoch 1/5 - Batch 5000/16000 Done - Train Loss: 0.457959, Val Loss: 0.461044
17:53:42 INFO   Epoch 1/5 - Batch 6000/16000 Done - Train Loss: 0.457823, Val Loss: 0.472640
17:53:52 INFO   Epoch 1/5 - Batch 7000/16000 Done - Train Loss: 0.457852, Val Loss: 0.465482
17:54:01 INFO   Epoch 1/5 - Batch 8000/16000 Done - Train Loss: 0.457886, Val Loss: 0.465019
17:54:11 INFO   Epoch 1/5 - Batch 9000/16000 Done - Train Loss: 0.457489, Val Loss: 0.462517
17:54:21 INFO   Epoch 1/5 - Batch 10000/16000 Done - Train Loss: 0.457241, Val Loss: 0.472444
17:54:30 INFO   Epoch 1/5 - Batch 11000/16000 Done - Train Loss: 0.45

## Model Performance

In [14]:
os.listdir(checkpoint_dir)

['FM_4', 'FM_1', 'FM_5', 'FM_2', 'FM_3']

In [15]:
model = FM_2D_Layer(len(embedding_map_dict)+60,39,5)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'FM_1')))
model.eval()
model = model.to(DEVICE)

In [16]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score(model, train_data, DEVICE)))

18:13:04 INFO   Model ROC Score: 0.805224


In [17]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score(model, test_data, DEVICE)))

18:13:47 INFO   Model ROC Score: 0.792564
